In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

# ---------------------------
# Load your dataset
# ---------------------------
# Example: data.csv -> each row is a spectrum, last column is the label
data = pd.read_csv("/content/synthetic_plastic_spectra.csv")

X = data.iloc[:, :-1].values   # all columns except last (intensities)
y = data.iloc[:, -1].values    # last column (label)

In [4]:
# Encode labels (PET->0, PE->1, ...)
le = LabelEncoder()
y = le.fit_transform(y)

# Normalize features
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [5]:
# Reshape for 1D CNN (samples, timesteps, channels)
X = X[..., np.newaxis]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# ---------------------------
# Build 1D CNN
# ---------------------------
model = models.Sequential([
    layers.Input(shape=(X.shape[1], 1)),
    layers.Conv1D(32, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(le.classes_), activation='softmax')
])

opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
# ---------------------------
# Train the model
# ---------------------------
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.8731 - loss: 0.8291 - val_accuracy: 1.0000 - val_loss: 0.0064
Epoch 2/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 1.0000 - loss: 0.0075 - val_accuracy: 1.0000 - val_loss: 6.0660e-04
Epoch 3/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 1.6074e-04
Epoch 4/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 1.0000 - loss: 5.2488e-04 - val_accuracy: 1.0000 - val_loss: 6.1127e-05
Epoch 5/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - accuracy: 1.0000 - loss: 2.7595e-04 - val_accuracy: 1.0000 - val_loss: 2.9385e-05
Epoch 6/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 1.0000 - loss: 1.7311e-04 - val_accuracy: 1.0000 - val_loss: 1.6262e-05
Epoch 7/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 1.0000 - loss: 1.2652e-04 - val_accuracy: 1.0000 - val_loss: 9.8117e-06
Epoch 8/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - 

In [8]:
# ---------------------------
# Evaluate
# ---------------------------
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print("Test Accuracy:", test_acc)

63/63 - 1s - 10ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy: 1.0
